## cv2 vs cv3

### Reading/Writing images

cv2

In [15]:
%%timeit

import cv2 as cv
import sys, os

img = cv.imread(cv.samples.findFile("img.jpeg"))
if img is None:
    sys.exit("Could not read the image.")

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

os.makedirs('outputs', exist_ok=True)
cv.imwrite('outputs/gray.jpg', gray)

3.21 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
import shutil
shutil.rmtree('outputs')

cv3

In [17]:
%%timeit

import cv3

img = cv3.imread('img.jpeg')
gray = cv3.rgb2gray(img)
cv3.imwrite('outputs/gray.jpg', gray, mkdir=True)

3.9 ms ± 76.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Drawing

cv2

In [18]:
%%timeit

import cv2 as cv

img = cv.imread(cv.samples.findFile("img.jpeg"))
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

cv.rectangle(img, (40, 30), (260, 220), (0, 0, 255), 3)
cv.putText(img, 'Parrot', (50, 25), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_8);

1.47 ms ± 13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


cv3

In [19]:
%%timeit

import cv3

img = cv3.imread('img.jpeg')

cv3.rectangle(img, 40, 30, 260, 220, color="blue", t=3)
cv3.text(img, "Parrot", 50, 25, color="white", t=2);

1.59 ms ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Reading/Writing video

cv2

In [34]:
%%timeit -n 30

import cv2 as cv

cap = cv.VideoCapture('vid.mp4')
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = None
i = 0
while cap.isOpened():
    i += 1
    if i == 5:
        break
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    if out is None:
        h, w = frame.shape[:2]
        out = cv.VideoWriter('output.avi', fourcc, 20.0, (w, h))
    frame = cv.flip(frame, 0)
    
    out.write(frame)
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
out.release()
cv.destroyAllWindows()

88.4 ms ± 3.33 ms per loop (mean ± std. dev. of 7 runs, 30 loops each)


cv3

In [35]:
%%timeit -n 30

import cv3

with (
	cv3.Video('vid.mp4') as cap,
	cv3.Video('output.avi', 'w', fps=20, fourcc='XVID') as out,
	cv3.Window('frame') as ww
):
    for i, frame in enumerate(cap):
        if i == 5:
            break
        frame = cv3.vflip(frame)
        out.write(frame)
        ww.imshow(frame)

        if cv3.wait_key(1) == ord('q'):
            break

103 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 30 loops each)


## Tasks in practice

In [2]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\rodion gorodenko\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import gdown

ids = [
    '1BieDvsslKpISGdbRbSECNSRfbXLozbe7', # zidane.jpg
    '15kty1_VBiqC6V27Lwdu654Rd22JJjBFE', # musk.jpg
    '1KGE_B37NY7Ag_UpEen-b0SgwRdUgjJHE', # cat.jpg
    '19SmL6tC8V6dsW7mHdqSi0yo6B1mj7mYy', # points.npy
    '1bC4XFNCL1jsCLCEge98uPIWXRmU3C1TR', # fractal.jpg
    # '1jzzOI1JLLmqJCOT4tvmWe7Ghk3HRI5-C', # shape_predictor_68_face_landmarks.dat
]

for id_ in ids:
    print(gdown.download(id=id_))

Downloading...
From: https://drive.google.com/uc?id=1BieDvsslKpISGdbRbSECNSRfbXLozbe7
To: C:\Users\Rodion Gorodenko\pycv\zidane.jpg
100%|███████████████████████████████████████████████████████████████████████████████| 169k/169k [00:00<00:00, 1.52MB/s]


zidane.jpg


Downloading...
From: https://drive.google.com/uc?id=15kty1_VBiqC6V27Lwdu654Rd22JJjBFE
To: C:\Users\Rodion Gorodenko\pycv\musk.jpg
100%|██████████████████████████████████████████████████████████████████████████████| 49.6k/49.6k [00:00<00:00, 681kB/s]


musk.jpg


Downloading...
From: https://drive.google.com/uc?id=1KGE_B37NY7Ag_UpEen-b0SgwRdUgjJHE
To: C:\Users\Rodion Gorodenko\pycv\cat.jpg
100%|████████████████████████████████████████████████████████████████████████████████| 119k/119k [00:00<00:00, 849kB/s]


cat.jpg


Downloading...
From: https://drive.google.com/uc?id=19SmL6tC8V6dsW7mHdqSi0yo6B1mj7mYy
To: C:\Users\Rodion Gorodenko\pycv\points.npy
100%|█████████████████████████████████████████████████████████████████████████████████████████| 592/592 [00:00<?, ?B/s]


points.npy


Downloading...
From: https://drive.google.com/uc?id=1bC4XFNCL1jsCLCEge98uPIWXRmU3C1TR
To: C:\Users\Rodion Gorodenko\pycv\fractal.jpg
100%|███████████████████████████████████████████████████████████████████████████████| 115k/115k [00:00<00:00, 1.01MB/s]

fractal.jpg


### YOLOv5 predictions visualization

In [2]:
import cv3
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
img = cv3.imread('zidane.jpg')
outputs = model([img])
results = outputs.pandas().xyxy[0]

for xmin, ymin, xmax, ymax, conf, cls_id, cls_name in results.values:
    cv3.rectangle(img, xmin, ymin, xmax, ymax, color=cv3.COLORS[cls_id], t=5)
    cv3.text(img, f'{cls_name} {conf:.2f}', xmin, ymin+30, color='white', t=2)

with cv3.Window() as wind:
    wind.imshow(img)
    wind.wait_key(0)

Using cache found in C:\Users\Rodion Gorodenko/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-15 Python-3.9.0 torch-2.0.1+cpu CPU

Fusing layers... 


requirements: C:\Users\Rodion Gorodenko\AppData\Local\Programs\Python\Python39\Lib\site-packages\requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


### Move images with processing

Creating folder with test images

In [3]:
from pathlib import Path
import shutil

Path('images').mkdir(exist_ok=True, parents=True)
for i in range(3):
    shutil.copy('img.jpeg', f'images/img{i}.jpeg')

Run code for moving and processing images

In [4]:
import cv3
from pathlib import Path

SRC_DIR = Path('images')
DST_DIR = Path('images256')
TARGET_SIZE = (256, 256)

for img_path in SRC_DIR.glob('*.jpeg'):
    img = cv3.imread(img_path)
    resized = cv3.resize(img, *TARGET_SIZE)
    dst_path = DST_DIR / img_path.name
    cv3.imwrite(dst_path, resized, mkdir=True)

Clean up directory

In [5]:
shutil.rmtree('images', ignore_errors=True)
shutil.rmtree('images256', ignore_errors=True)

### dlib detections visualization

In [6]:
import cv3
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

img = cv3.imread('musk.jpg')
dets = detector(img)

for det in dets:
    cv3.rectangle(img, det.left(), det.top(), det.right(), det.bottom(), t=2)
    shape = predictor(img, det)
    for point in shape.parts():
        cv3.point(img, point.x, point.y, color='lime', r=2)

with cv3.Window() as wind:
    wind.imshow(img)
    wind.wait_key(0)

### Draw annotatation with polygons

In [7]:
import numpy as np

YELLOW = np.array([255, 255, 0])

img_path = 'cat.jpg'
label = 'cat'

points = np.load('points.npy')

In [8]:
import cv3

img = cv3.imread(img_path)

# polylines
annotated = cv3.polylines(img, points, is_closed=True, t=5, copy=True)

# fill polylines
mask = cv3.zeros(*img.shape[:2])
cv3.fill_poly(mask, points, color="red")
annotated[mask > 0] = 0.5 * YELLOW + 0.5 * annotated[mask > 0]

with cv3.Window("Annotated example") as wind:
    wind.imshow(annotated)
    wind.wait_key(0)

### Display video stream in many windows

In [1]:
import cv3

with cv3.Windows(['Original', 'Gray', 'Threshold', 'HSV']) as ws:
    for frame in cv3.Video(0):
        gray = cv3.rgb2gray(frame)
        thr = cv3.threshold(gray)
        hsv = cv3.rgb2hsv(frame)

        ws['Original'].imshow(frame)
        ws['Gray'].imshow(gray)
        ws['Threshold'].imshow(thr)
        ws['HSV'].imshow(hsv)
        if cv3.wait_key(1) == ord('q'):
            break

### Video processing

In [10]:
import cv3

with cv3.Video('output.mp4', 'w') as out:
    with cv3.Video('vid.mp4') as cap:
        for frame in cap:
            gray = cv3.rgb2gray(frame)
            out.write(gray)

### Creating animation

In [11]:
import cv3
import numpy as np

n_iter = 100
img_orig = cv3.imread('fractal.jpg')
scales = np.linspace(0.3, 1.7, n_iter)
with cv3.Video('fractal.mp4', 'w') as out:
    for i in range(n_iter):
        img = cv3.transform(img_orig, scale=scales[i], angle=i, border='default')
        out.write(img)